In [ ]:
"""
ResNet50-based model for material property prediction.
This module implements a ResNet50-based model for predicting material properties.
"""

import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
from tqdm import tqdm
import os
import subprocess


In [ ]:

def get_least_utilized_gpu():
    """Get the GPU with the least memory usage."""
    try:
        # Run nvidia-smi to get GPU memory usage
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
                              capture_output=True, text=True)
        memory_usage = [int(x) for x in result.stdout.strip().split('\n')]
        
        # Find GPU with minimum memory usage
        min_usage = min(memory_usage)
        gpu_id = memory_usage.index(min_usage)
        
        print(f"Selected GPU {gpu_id} with {min_usage}MB memory usage")
        return gpu_id
    except Exception as e:
        print(f"Error getting GPU usage: {e}")
        return 0  # Fallback to GPU 0 if there's an error


In [ ]:

class MaterialDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.FloatTensor(features)
        self.targets = torch.FloatTensor(targets)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]
    
class MaterialPropertyMLP(nn.Module):
    def __init__(self, input_dim, output_dim=1):
        super(MaterialPropertyMLP, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )
    
    def forward(self, x):
        return self.fc(x)
 

In [ ]:
   
class ResNet50Model(nn.Module):
    def __init__(self, input_dim, output_dim=1):
        super(ResNet50Model, self).__init__()
        
        # Load pre-trained ResNet50
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        # Modify the first layer to accept our input dimension
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Modify the final layer for our regression task
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, output_dim)
        )
        
    def forward(self, x):
        # Reshape input to match ResNet50's expected input shape
        batch_size = x.size(0)
        x = x.view(batch_size, 1, -1, 1)  # Reshape to [batch_size, 1, features, 1]
        x = x.expand(-1, 1, -1, 224)  # Expand to match ResNet50's expected size
        return self.resnet(x)


In [ ]:

def train_resnet50(X_train, X_test, y_train, y_test, target='band_gap', 
                  batch_size=32, num_epochs=50, learning_rate=0.001):
    """
    Train a ResNet50-based model for material property prediction.
    
    Args:
        X_train, X_test: Training and testing features
        y_train, y_test: Training and testing targets
        target: Target property name
        batch_size: Batch size for training
        num_epochs: Number of training epochs
        learning_rate: Learning rate for optimizer
    """
    # Select the least utilized GPU
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "backend:cudaMallocAsync"

    gpu_id = get_least_utilized_gpu()
    device = torch.device(f'cuda:{gpu_id}')
    print(f"Using GPU {gpu_id} for training")
    torch.cuda.empty_cache()
    # Create datasets
    train_dataset = MaterialDataset(X_train, y_train)
    test_dataset = MaterialDataset(X_test, y_test)
    
    # Create data loaders with multiple workers
    num_workers = 8#min(4, os.cpu_count() or 1)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                            num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                           num_workers=num_workers, pin_memory=True)
    
    # Initialize model
    model = ResNet50Model(input_dim=X_train.shape[1])
    #model = MaterialPropertyMLP(input_dim=X_train.shape[1])
    model = model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_losses = []
    test_losses = []
    
    # Create progress bar for epochs
    epoch_pbar = tqdm(range(num_epochs), desc="Training Progress")
    
    for epoch in epoch_pbar:
        model.train()
        running_loss = 0.0
        # Create progress bar for batches
        batch_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        
        for inputs, targets in batch_pbar:
            inputs, targets = inputs.cuda(), targets.cuda()
            optimizer.zero_grad()
            print(torch.cuda.memory_summary())
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            batch_pbar.set_postfix({'batch_loss': f'{loss.item():.4f}'})
        
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)
        # Evaluate on test set
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                test_loss += criterion(outputs, targets).item()
        
        test_loss = test_loss / len(test_loader)
        test_losses.append(test_loss)
        
        # Update epoch progress bar
        epoch_pbar.set_postfix({
            'train_loss': f'{train_loss:.4f}',
            'test_loss': f'{test_loss:.4f}'
        })
    
    # Save the model
    torch.save(model.state_dict(), f'models/resnet50_{target}.pth')
    
    return model, train_losses, test_losses


In [ ]:

def predict_resnet50(model, X):
    """
    Make predictions using the trained ResNet50 model.
    
    Args:
        model: Trained ResNet50 model
        X: Input features
    """
    model.eval()
    gpu_id = get_least_utilized_gpu()
    device = torch.device(f'cuda:{gpu_id}')
    model = model.to(device)
    
    with torch.no_grad():
        X = torch.FloatTensor(X).to(device)
        predictions = model(X)
    
    return predictions.cpu().numpy() 

In [ ]:
"""
Script to train the ResNet50 model for material property prediction.
This script uses the existing data preprocessing pipeline and trains the ResNet50 model.
"""

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from resnet50_model import train_resnet50, predict_resnet50
import joblib
from collections import Counter


In [ ]:

def convert_elements_to_features(elements_str):
    """Convert elements string into numeric features"""
    # Split elements and count occurrences
    elements = elements_str.split(',')
    element_counts = Counter(elements)
    
    # Create a dictionary of element counts
    features = {}
    for element in element_counts:
        features[f'element_{element}'] = element_counts[element]
    
    return features


In [ ]:

def load_and_preprocess_data(target='band_gap'):
    """Load and preprocess data for ResNet50 model"""
    # Load data
    df = pd.read_csv("materials_for_ml.csv")
    
    # Drop rows with missing target values
    df = df.dropna(subset=[target])
    
    # Convert elements to numeric features
    element_features = pd.DataFrame([convert_elements_to_features(elem) for elem in df['formula']])
    element_features = element_features.fillna(0)  # Fill NaN with 0 for elements not present
    
    # Select features
    categorical_cols = ["crystal_system"]
    element_cols = [col for col in df.columns if col.startswith('contains_')]
    numerical_cols = [col for col in df.columns if col not in categorical_cols + element_cols + [target, 'material_id', 'formula']]
    
    # One-hot encode categorical features
    df = pd.get_dummies(df, columns=categorical_cols)
    
    # Prepare features and target
    X = df.drop(columns=[target, 'material_id', 'formula'])
    
    # Combine original features with element features
    X = pd.concat([X, element_features], axis=1)
    
    y = df[target]
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Save the scaler
    joblib.dump(scaler, f'models/resnet50_{target}_scaler.pkl')
    
    return X_train_scaled, X_test_scaled, y_train.values, y_test.values


In [ ]:

def plot_training_history(train_losses, test_losses, target):
    """Plot training and validation loss history"""
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(test_losses, label='Validation Loss')
    plt.title(f'ResNet50 Training History - {target}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'plots/resnet50_{target}_training_history.png')
    plt.close()


In [ ]:

def main():
    # Target properties to predict
    targets = ['band_gap', 'formation_energy_per_atom']  # Add more targets as needed
    
    for target in targets:
        print(f"\nTraining ResNet50 model for {target}...")
        
        # Load and preprocess data
        X_train, X_test, y_train, y_test = load_and_preprocess_data(target)
        
        # Train model
        model, train_losses, test_losses = train_resnet50(
            X_train, X_test, y_train, y_test,
            target=target,
            batch_size=32,
            num_epochs=5,
            learning_rate=0.001
        )
        
        # Plot training history
        plot_training_history(train_losses, test_losses, target)
        
        # Make predictions
        y_pred = predict_resnet50(model, X_test)
        
        # Calculate and print metrics
        mse = np.mean((y_test - y_pred) ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(y_test - y_pred))
        
        print(f"\nResults for {target}:")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")



In [ ]:
main() 